In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
!pip install matplotlib opencv-python Pillow PyYAML requests scipy tqdm tensorboard pandas seaborn ipython psutil thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00


In [4]:
! git clone https://github.com/BlankCode0/yolov7-mod.git

Cloning into 'yolov7-mod'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 178 (delta 1), reused 0 (delta 0), pack-reused 169 (from 2)
Receiving objects: 100% (178/178), 293.33 MiB | 28.31 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Updating files: 100% (138/138), done.


In [5]:
# Upload kaggle.json then download dataset (one-time)
from google.colab import files
import os, shutil, subprocess

uploaded = files.upload()  # choose kaggle.json
if 'kaggle.json' in uploaded:
    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    os.chmod('/root/.kaggle/kaggle.json', 0o600)
    print("kaggle.json installed.")
else:
    raise SystemExit("Please upload kaggle.json and re-run this cell.")

# install kaggle and download the YOLO-ready vehicle dataset
!pip install -q kaggle
!kaggle datasets download -d muhammadhananasghar/vehicle-detectionyolo-version -p /content/vehicle_ds --unzip
!ls -lh /content/vehicle_ds


Saving kaggle.json to kaggle.json
kaggle.json installed.
Dataset URL: https://www.kaggle.com/datasets/muhammadhananasghar/vehicle-detectionyolo-version
License(s): CC0-1.0
 82% 250M/304M [00:00<00:00, 559MB/s]
100% 304M/304M [00:00<00:00, 599MB/s]
total 68K
drwxr-xr-x 2 root root 68K Sep  7 07:03 'Vehicle Detection'


In [8]:
# Run this in Colab. Adjust SRC if your folder name differs slightly.
import os, glob, random, shutil

random.seed(42)

SRC = '/content/vehicle_ds/Vehicle Detection/'   # your folder with 1.jpg 1.txt etc
OUT = '/content/vehicle_ds_800'

# find images (jpg/png) and ensure matching .txt exists
img_files = sorted(glob.glob(os.path.join(SRC, '*.jpg')) + glob.glob(os.path.join(SRC, '*.png')))
pairs = []
for img in img_files:
    base = os.path.splitext(os.path.basename(img))[0]
    lbl = os.path.join(SRC, base + '.txt')
    if os.path.exists(lbl) and os.path.getsize(lbl) > 0:
        pairs.append((img, lbl))

if len(pairs) == 0:
    raise SystemExit(f"No labelled images found in {SRC} — check path and file extensions.")

N_want = 800
if len(pairs) < N_want:
    print(f"Only {len(pairs)} labelled images available — will use all of them.")
    chosen = pairs
else:
    chosen = random.sample(pairs, N_want)

# prepare output dirs
for split in ['train','val']:
    os.makedirs(os.path.join(OUT,'images',split), exist_ok=True)
    os.makedirs(os.path.join(OUT,'labels',split), exist_ok=True)

# split 80/20
n = len(chosen)
n_train = int(n * 0.8)
train = chosen[:n_train]
val = chosen[n_train:]

# copy files
for img, lbl in train:
    shutil.copy(img, os.path.join(OUT,'images','train', os.path.basename(img)))
    shutil.copy(lbl, os.path.join(OUT,'labels','train', os.path.basename(lbl)))
for img, lbl in val:
    shutil.copy(img, os.path.join(OUT,'images','val', os.path.basename(img)))
    shutil.copy(lbl, os.path.join(OUT,'labels','val', os.path.basename(lbl)))

print(f"Done — created {OUT} with {len(train)} train and {len(val)} val images.")
print("Sample train files:", os.listdir(os.path.join(OUT,'images','train'))[:5])
print("Sample val files:", os.listdir(os.path.join(OUT,'images','val'))[:5])

Done — created /content/vehicle_ds_800 with 640 train and 160 val images.
Sample train files: ['121.jpg', '1211.jpg', '356.jpg', '2560.jpg', '309.jpg']
Sample val files: ['2816.jpg', '2530.jpg', '2797.jpg', '392.jpg', '2798.jpg']


In [9]:
%cd /content/yolov7-mod

/content/yolov7-mod


In [10]:

# %cp /content/yolov7-mod/data/coco.yaml /content/yolov7-mod/data/custom.yaml
!wget "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt"

--2025-09-07 07:10:57--  https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/511187726/b0243edf-9fb0-4337-95e1-42555f1b37cf?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-07T07%3A54%3A24Z&rscd=attachment%3B+filename%3Dyolov7.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-07T06%3A53%3A35Z&ske=2025-09-07T07%3A54%3A24Z&sks=b&skv=2018-11-09&sig=O9N%2B2foS4MaKKN9jfTzIrXzswpBh2Nh3rY5X225XoBs%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NzIyOTM1NywibmJmIjoxNzU3MjI5MDU3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcmUud2lu

In [ ]:
# %cp /content/yolov7-mod/cfg/training/yolov7.yaml /content/yolov7-mod/cfg/training/custom_yolov7.yaml

tuning this on trains the model but issue at the end

In [11]:
!sed -i "s/torch.load(weights, map_location=device)/torch.load(weights, map_location=device, weights_only=False)/g" train.py

In [12]:
!sed -i "s/torch.load(weights, map_location=device)/torch.load(weights, map_location=device, weights_only=False)/g" utils/general.py
!sed -i "s/torch.load(f, map_location=torch.device('cpu'))/torch.load(f, map_location=torch.device('cpu'), weights_only=False)/g" utils/general.py

In [13]:
!sed -i "s/torch.load(cache_path)/torch.load(cache_path, weights_only=False)/g" utils/datasets.py

In [23]:
%cd /content/yolov7-mod
!ls -lh yolov7.pt   # must see the file and size here

!python train.py \
  --batch 8 \
  --cfg cfg/training/yolov7-mod.yaml \
  --epochs 30 \
  --data data/custom.yaml \
  --weights /content/yolov7-mod/yolov7.pt\ # change this code according to the path of your pretrained weights
  --device 0


/content/yolov7-mod
-rw-r--r-- 1 root root 73M Jul  6  2022 yolov7.pt
2025-09-07 09:08:19.755899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757236099.791313   34910 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757236099.802298   34910 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757236099.828145   34910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757236099.828183   34910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17572

In [32]:
%cd /content/yolov7-mod

# 1) Patch models/experimental.py (attempt_load)
!sed -i "s/torch.load(w, map_location=map_location)/torch.load(w, map_location=map_location, weights_only=False)/g" models/experimental.py

# 2) (Recommended) Patch other possible torch.load callsites too:
!sed -i "s/torch.load(weights, map_location=device)/torch.load(weights, map_location=device, weights_only=False)/g" train.py
!sed -i "s/torch.load(f, map_location=torch.device('cpu'))/torch.load(f, map_location=torch.device('cpu'), weights_only=False)/g" utils/general.py
!sed -i "s/torch.load(cache_path)/torch.load(cache_path, weights_only=False)/g" utils/datasets.py

# 3) Run detect with no tracing (to avoid the earlier FP16/tracing issue)
!python detect.py \
  --weights /content/yolov7-mod/runs/train/exp6/weights/best.pt \
  --img-size 640 \
  --conf 0.36 \
  --source /content/vehicle_ds_800/images/train/10.jpg \
  --no-trace

/content/yolov7-mod
Namespace(weights=['/content/yolov7-mod/runs/train/exp6/weights/best.pt'], source='/content/vehicle_ds_800/images/train/10.jpg', img_size=640, conf_thres=0.36, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=True)
YOLOR 🚀 8b6b0e5 torch 2.8.0+cu126 CUDA:0 (Tesla T4, 15095.0625MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 215 layers, 43338210 parameters, 4720384 gradients, 170.8 GFLOPS
5 cars, Done. (24.4ms) Infe

In [33]:
%cd /content/yolov7-mod

# 1) Patch models/experimental.py (attempt_load)
!sed -i "s/torch.load(w, map_location=map_location)/torch.load(w, map_location=map_location, weights_only=False)/g" models/experimental.py

# 2) (Recommended) Patch other possible torch.load callsites too:
!sed -i "s/torch.load(weights, map_location=device)/torch.load(weights, map_location=device, weights_only=False)/g" train.py
!sed -i "s/torch.load(f, map_location=torch.device('cpu'))/torch.load(f, map_location=torch.device('cpu'), weights_only=False)/g" utils/general.py
!sed -i "s/torch.load(cache_path)/torch.load(cache_path, weights_only=False)/g" utils/datasets.py

# 3) Run detect with no tracing (to avoid the earlier FP16/tracing issue)
!python detect.py \
  --weights /content/yolov7-mod/yolov7.pt \
  --img-size 640 \
  --conf 0.3 \
  --source /content/vehicle_ds_800/images/train/10.jpg \
  --no-trace

/content/yolov7-mod
Namespace(weights=['/content/yolov7-mod/yolov7.pt'], source='/content/vehicle_ds_800/images/train/10.jpg', img_size=640, conf_thres=0.3, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=True)
YOLOR 🚀 8b6b0e5 torch 2.8.0+cu126 CUDA:0 (Tesla T4, 15095.0625MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 306 layers, 36905341 parameters, 6652669 gradients, 104.5 GFLOPS
12 cars, 2 trucks, Done. (15.9ms) Inference, (60.6ms) NMS
 The 